In [1]:
from game2048.game import Game
from game2048.displays import Display, IPythonDisplay
from game2048.agents import Agent, RandomAgent, ExpectiMaxAgent
# from game2048.displayer import Displayer
display1 = Display()
display2 = IPythonDisplay()

In [2]:
import numpy as np
import datetime
import csv
import math

map_table = {2**i : i for i in range(1,16)}
map_table[0] = 0

class DataAgent(ExpectiMaxAgent):
    def auto_log(self, data_dir="./data/", max_iter=np.inf):
        filename = data_dir + datetime.datetime.now().strftime('%y%m%d_%H%M%S_%f') + ".csv"
        print("文件保存到：",filename,end='\t')
        with open(filename,"w") as csvfile: 
            writer = csv.writer(csvfile)
            n_iter = 0
            while (n_iter < max_iter) and (not self.game.end):
                direction = self.step()
                bd = list(self.game.board.flatten())
                bd = [int(s) for s in bd]
                bd = [map_table[i] for i in bd]
                #bd = [int(math.log(i,2)) if i>0 else i for i in bd]
                bd.append(direction)
                writer.writerow(bd)
                
                # 模拟当前正确率 0.72 = x + (1-x)/4 => x = 0.63
                if(random.random()>0.63):
                    direction = random.randrange(4)
                self.game.move(direction)
                n_iter += 1
        print("数据量：",n_iter)
        

In [ ]:
# 生成数据，一局一个csv文件，无header无index，每行数据为 [log2后的棋盘, 下一步的方向]
for i in range(1000):
    game = Game(4, score_to_win=2048, random=False)
    agent = DataAgent(game, display=None)
    print("#%d\t"%i,end=' ')
    agent.auto_log()

In [6]:
#聚合path下所有csv
# 有时候可能目录下会有.ipynb_checkpoints，请手动删除
import os
import datetime

path = './data/'
files = os.listdir(path)
outputfile = "./" + "all_%d_%s.csv"%(len(files),datetime.datetime.now().strftime('%y%m%d_%H%M%S'))
print("文件保存到：", outputfile)  # 文件默认不保存到data目录

target = open(outputfile,"w")
for file in files: 
    with open(path+file, "r") as f:
        for line in f:
            target.write(line)
target.close()

文件保存到： ./all_10_181216_235355.csv


In [155]:
count = 0
with open(outputfile,"r") as f:
    for line in f:
        count +=1
count

14055

In [4]:
import numpy as np
import datetime
import csv
import random

map_table = {2**i : i for i in range(1,16)}
map_table[0] = 0

class BigDataAgent(ExpectiMaxAgent):
    def auto_log(self, data_dir="./data/", max_iter=1000, acc=1):
        filename = data_dir + datetime.datetime.now().strftime('%y%m%d_%H%M%S_%f') + ".csv"
        print("文件保存到：",filename)
        acc_th = (4*acc-1)/3  # 模拟当前正确率
        with open(filename,"w") as csvfile: 
            writer = csv.writer(csvfile)
            n_iter = 0
            n_run = 0
            while (n_iter < max_iter):
                if(n_iter%10000==0):
                        print("局数：",n_run,"目前数据量：",n_iter)
                if self.game.end:
                    n_run += 1
                    self.game = Game(4, score_to_win=2048, random=False)
                direction = self.step()
                bd = list(self.game.board.flatten())
                bd = [int(s) for s in bd]
                bd = [map_table[i] for i in bd]
                bd.append(direction)
                writer.writerow(bd)
                
                # 模拟当前正确率 0.72 = x + (1-x)/4 => x = 0.63
                if(random.random()>acc_th):
                    direction = random.randrange(4)
                self.game.move(direction)
                n_iter += 1
        print("数据量：",n_iter)
        

In [ ]:
# 生成数据，一局一个csv文件，无header无index，每行数据为 [log2后的棋盘, 下一步的方向]
game = Game(4, score_to_win=2048, random=False)
agent = BigDataAgent(game, display=None)
for i in range(1):
    agent.auto_log(max_iter=1000000,acc=1)

In [6]:
from game2048.game import Game
from game2048.displays import Display, IPythonDisplay
from game2048.agents import Agent, RandomAgent, ExpectiMaxAgent
import math
import numpy as np

In [17]:
display1 = Display()
display2 = IPythonDisplay()

In [113]:
game = Game(4, random=False)
game.enable_rewrite_board = True
cur_board = game.board
cur_board

array([[0., 2., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 4., 0., 0.]])

In [114]:
'''
direction:
    0: left
    1: down
    2: right
    3: up
'''
comment = ["left","down","right","up"]
for d in range(4):
    print(comment[d])
    game.board = cur_board
    game.move(d)
    print(game.board)
    #display2.display(game)

left
[[2. 0. 0. 0.]
 [2. 0. 0. 0.]
 [0. 0. 0. 0.]
 [4. 0. 0. 0.]]
down
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 2. 0. 2.]
 [0. 4. 0. 0.]]
right
[[0. 0. 0. 2.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 4. 0. 4.]]
up
[[0. 2. 0. 0.]
 [0. 4. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 2. 0.]]


In [115]:
game.board = cur_board
bd = list(game.board.flatten())
bd = [int(s) for s in bd]
print(bd)
bd = [int(math.log(i,2)) if i>0 else i for i in bd]
bd
#bd.append(direction)

[0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0]


[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0]

In [156]:
line = [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 3]

In [157]:
bd = [2<<(i-1) if i>0 else 0 for i in line[:-1]]
bd

[0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0]

In [158]:
cur_board = np.array(bd).reshape(4,4)
cur_board

array([[0, 2, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 4, 0, 0]])

In [159]:
comment = ["left","down","right","up"]
buf = []
for d in range(4):
    #print(comment[d])
    game.board = cur_board
    game.move(d)
    buf.append(game.board.astype(int))
    print(buf[d])

[[2 0 0 0]
 [0 0 4 0]
 [0 0 0 0]
 [4 0 0 0]]
[[0 0 0 0]
 [0 0 0 0]
 [0 2 0 0]
 [2 4 0 0]]
[[0 0 0 2]
 [0 0 0 4]
 [0 0 0 0]
 [0 0 0 4]]
[[0 2 0 4]
 [0 4 0 0]
 [0 0 0 0]
 [0 0 0 0]]


In [203]:
#构建映射表
'''
[0:left, 3:up   ]
[1:down, 2:right]
'''
tmp = np.vstack((np.hstack((buf[0],buf[3])) ,np.hstack((buf[1],buf[2]))))

In [223]:
new = np.append(tmp,line[-1]).tolist()


In [205]:
print(new)

['8', '0', '0', '0', '4', '0', '0', '4', '0', '0', '0', '0', '0', '2', '0', '0', '4', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '8', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '4', '0', '4', '4', '4', '0', '0', '0', '\n']


In [193]:
len(new)

65

In [8]:
game = Game(4, random=False)
game.enable_rewrite_board = True
cur_board = game.board
cur_board

array([[0., 0., 0., 2.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 2., 0., 0.]])

In [9]:
with open("datatrain2M.csv","w") as target:
    writer = csv.writer(target)
    with open("data2M.csv","r") as f:
        for line in f:
            t = eval(line)
            bd = [2<<(i-1) if i>0 else 0 for i in t[:-1]]
            cur_board = np.array(bd).reshape(4,4)
            buf = []
            for d in range(4):
                #print(comment[d])
                game.board = cur_board
                game.move(d)
                buf.append(game.board.astype(int))
                #print(buf[d])
            tmp = np.vstack((np.hstack((buf[0],buf[3])) ,np.hstack((buf[1],buf[2])))).flatten().tolist()
            new = [int(math.log(i,2)) if i>0 else i for i in tmp]
            new.append(t[-1])
            #print(new)
            writer.writerow(new)

In [2]:
import csv

In [234]:
len(new)

65

In [ ]:
import csv
model = keras.models.load_model('model_k.h5')

for it in range(3):
    for index in range(6):
        data = []
        with open("./train/train1M_%d.csv"%(index+1)) as f:
            for line in f:
                piece = eval(line)
                data.append(piece)

        data = np.array(data)

        x = np.array([ grid_one(piece[:-1].reshape(4,4)) for piece in data ])
        y = keras.utils.to_categorical(data[:,-1], 4)

        sep = 1000000
        x_train = x[:sep]
        x_test = x[sep:]
        y_train = y[:sep]
        y_test = y[sep:]

        model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=10)
        
        model.save('model_k.h5')


In [ ]:
import csv
data = []
with open("./train/train1M_2.csv") as f:
    for line in f:
        piece = eval(line)
        data.append(piece)

data = np.array(data)

x = np.array([ grid_one(piece[:-1].reshape(4,4)) for piece in data ])
y = keras.utils.to_categorical(data[:,-1], 4)

sep = 900000
x_train = x[:sep]
x_test = x[sep:]
y_train = y[:sep]
y_test = y[sep:]


model = keras.models.load_model('model_k.h5')

# train , validation_data=(x_test,y_test)
model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=30)

score_test = model.evaluate(x_test,y_test,verbose=0)
print('Testing loss: %.4f, Testing accuracy: %.2f' % (score_test[0],score_test[1]))